<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>

You might want to consider the [start](search.ipynb) of this tutorial.

Short introductions to other TF datasets:

* [Dead Sea Scrolls](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/lorentz2020/dss.ipynb),
* [Old Babylonian Letters](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/lorentz2020/oldbabylonian.ipynb),
or the
* [Q'uran](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/lorentz2020/quran.ipynb)


# Upgrade features along a node mapping

Consider the semantic actor features in 
[ch-jensen/participants/actor/tf](https://github.com/ch-jensen/participants/tree/master/actor/tf).

We see only features for version `c` of the BHSA, but we prefer to work with version `2021` of the BHSA.

When we try to load the features by simply saying

```
A = use("bhsa", mod="ch-jensen/participants/actor/tf")
```

we have no luck, because there is no `ch-jensen/participants/actor/tf/2021` on Github.

But, one of the features in the BHSA is `omap@c-2021.tf` and this contains the information to map
all nodes in version `c` to the nodes of version `2021`, as faithfully as is reasonably possible.

My homework as Text-Fabric developer is to make it so that the statement above works, by steering Text-Fabric
to download version `c` and using the mapping feature to produce upgraded data in the right place.
But I have not get round to that yet.

So, here is what *you* can do about it 😎.

1. File an [issue](https://github.com/ch-jensen/participants/issues) and ask Christian whether he is inclined to
   use his software to build the features against BHSA version 2021.
   *But he might be too busy to do that right now.*
2. Fork [ch-jensen/participants](https://github.com/ch-jensen/participants) and try to run his software yourself.
   *That might not be easy. It seems that the code to run is in another repository.
   Is all the input data publicly available? Are special settings needed for version 2021?
   Is the software still executable?*
3. Do fork the repo by all means, and then use a tool of text-fabric to *upgrade* the features of the older version
   to the newer version.
   
We take you through the last option and evaluate how well the upgrade process fares.

In [1]:
%load_ext autoreload
%autoreload 2

# Incantation

The ins and outs of installing Text-Fabric, getting the corpus, and initializing a notebook are
explained in the [start tutorial](start.ipynb).

In [2]:
import collections

from tf.app import use
from tf.fabric import Fabric
from tf.dataset.nodemaps import Versions

## Load the current version of the BHSA

We need the current version (`2021`) of the BHSA anyway, so we are going to load it.

## Convention

We will have two versions of the corpus in our notebook and in our variables.
It is handy to have a consistent naming scheme:

* `N` (the *now* version): `2021`
* `P` (the *previous* version): `c`

In [3]:
N = use("bhsa")

This is Text-Fabric 9.1.11
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


## Load the available version of the participant features

We have forked Christian's repo to `etcbc/participants`, so make sure to clone it to your computer:

```
cd ~/github/etcbc
git clone https://github.com/ETCBC/participants
```

In [4]:
LOCATION = "~/github/etcbc/participants/actor/tf"

Now we can load the actor features for version `c`.

In [5]:
P = use(LOCATION, version="c")

This is Text-Fabric 9.1.11
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

3 features found and 0 ignored
  0.00s Not all of the warp features otype and oslots are present in
~/github/etcbc/participants/actor/tf/c
  0.00s Only the Feature and Edge APIs will be enabled
  0.00s Warp feature "otext" not found. Working without Text-API



By clicking the triangles you can find more information about these features.

## Upgrade the participant features

We are going to upgrade the participant features from version `c` to version `2021`.

For that, we use [tf.dataset.nodemaps.Versions](https://annotation.github.io/text-fabric/tf/dataset/nodemaps.html#tf.dataset.nodemaps.Versions).

We initialize the Versions object with two text-fabric api objects:

In [6]:
apis = {"2021": N.api, "c": P.api}

V = Versions(apis, "c", "2021")

Finally we migrate the features from "c" to "2021" and save them in the correct location.

We skip the `otext` feature, since it is a special config feature, not a data feature made by Christian.

In [7]:
V.migrateFeatures(("actor", "coref", "prs_actor"), location=LOCATION)

  0.00s Exporting 2 node and 1 edge and 0 config features to ~/github/etcbc/participants/actor/tf/2021:
   |     0.00s T actor                to ~/github/etcbc/participants/actor/tf/2021
   |     0.00s T prs_actor            to ~/github/etcbc/participants/actor/tf/2021
   |     0.05s T coref                to ~/github/etcbc/participants/actor/tf/2021
  0.06s Exported 2 node features and 1 edge features and 0 config features to ~/github/etcbc/participants/actor/tf/2021


## Load the upgraded module

Now we are in a position that we can load version 2021 of the BHSA together with the migrated module of participant features.
Note that we we point Text-Fabric to the forked repo (`etcbc` instead of `ch-jensen`) and then to
our local clone (`:clone`).

In [8]:
N = use("bhsa", mod="etcbc/participants/actor/tf:clone")

This is Text-Fabric 9.1.11
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

125 features found and 0 ignored
   |     0.02s T actor                from ~/github/etcbc/participants/actor/tf/2021
   |     0.26s T coref                from ~/github/etcbc/participants/actor/tf/2021
   |     0.01s T prs_actor            from ~/github/etcbc/participants/actor/tf/2021


If you click the triangles and navigate to the full metadata of the participants features,
you see a line

```
upgraded: ‼️ from version c to 2021
```

## Checks

Let's do a few checks to see how well the upgrade process has worked.

First we load the `c` version of the BHSA and Christian's original features.

In [9]:
P = use("bhsa", mod="ch-jensen/participants/actor/tf", version="c")

This is Text-Fabric 9.1.11
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

123 features found and 0 ignored


Below we are going to peek into the corpus by means of pretty dispays.
Here we tweak what is displayed and in what style.

*  we load the node mapping feature since it is not loaded by default
*  we hide a few container types that are not relevant for our investigation
*  we display material in sentence containers
*  we use the phonological transcription, instead of fully pointed Hebrew,
   so that non-Hebraists can see what is happening here.

In [10]:
N.load("omap@c-2021", silent=True)
N.isLoaded("omap@c-2021")

hiddenTypes="half_verse,sentence_atom,clause,clause_atom"

N.displaySetup(hiddenTypes=hiddenTypes, condenseType="sentence", withNodes=True, fmt="text-phono-full")
P.displaySetup(hiddenTypes=hiddenTypes, condenseType="sentence", withNodes=True, fmt="text-phono-full")

omap@c-2021          edge (int) ⚠️ Maps the nodes of version c to 2021


### Node feature "actor"

What are the node types that have an *actor* value?

In [11]:
{P.api.F.otype.v(n) for n in P.api.N.walk() if P.api.F.actor.v(n) is not None}

{'phrase_atom', 'subphrase'}

In [12]:
{N.api.F.otype.v(n) for n in N.api.N.walk() if N.api.F.actor.v(n) is not None}

{'phrase_atom', 'subphrase'}

Let's inspect the frequency lists of *actor*, per node type.

In [13]:
for otype in ("phrase_atom", "subphrase"):
    frequenciesN = N.api.F.actor.freqList(nodeTypes={otype})
    frequenciesP = P.api.F.actor.freqList(nodeTypes={otype})
    freqDictN = {v: f for (v, f) in frequenciesN}
    freqDictP = {v: f for (v, f) in frequenciesP}
    goodOnes = []
    badOnes = []
    for v in sorted(set(freqDictN) | set(freqDictP)):
        fN = freqDictN.get(v, 0)
        fP = freqDictP.get(v, 0)
        if fN == fP:
            goodOnes.append(v)
        else:
            badOnes.append((v, fN, fP))
            
    print(f"\nComparing frequencies on {otype}s: {len(goodOnes)} OK; {len(badOnes)} discrepancies")
    for (v, fN, fP) in badOnes[0:100]:
        print(f"{fN:>3} {fP:>3} {v}")


Comparing frequencies on phrase_atoms: 361 OK; 2 discrepancies
 91  94 >JC
  7   9 CNH

Comparing frequencies on subphrases: 135 OK; 0 discrepancies


### Closer inspection

Most actors on phrase atoms carry over well. But e.g. `CNH` has discrepancies.
Let's get a feel of why we get the discrepancies.

In [14]:
actorCNH = """
phrase_atom
  actor=CNH
"""

resultsN = N.search(actorCNH)
resultsP = P.search(actorCNH)

  0.18s 7 results
  0.17s 9 results


In [15]:
N.table(resultsN)
P.table(resultsP)

n,p,phrase_atom
1,Leviticus 25:11,945873tihyˈeh
2,Leviticus 25:12,945886yôvˈēl
3,Leviticus 25:12,945887hˈiw
4,Leviticus 25:12,945888qˌōḏeš
5,Leviticus 25:12,945889tihyˈeh
6,Leviticus 25:51,946353baššānˈîm
7,Leviticus 25:52,946362baššānˈîm


n,p,phrase_atom
1,Leviticus 25:10,945830šānˈā
2,Leviticus 25:11,945851šānˌā
3,Leviticus 25:11,945852tihyˈeh
4,Leviticus 25:12,945865yôvˈēl
5,Leviticus 25:12,945866hˈiw
6,Leviticus 25:12,945867qˌōḏeš
7,Leviticus 25:12,945868tihyˈeh
8,Leviticus 25:51,946332baššānˈîm
9,Leviticus 25:52,946341baššānˈîm


Clearly, there is something interesting in Leviticus 25 verses 10 and 11.

We compare verse 10 in both versions.
Here are the original actors in version `c`:

In [16]:
P.show(resultsP, start=1, end=1, condensed=True)

Let's find the same sentence in version `2021`

In [17]:
sP = 1181939
mappedSb = N.api.Es("omap@c-2021").f(sP)
mappedSb

((1181957, None),)

In [18]:
N.pretty(mappedSb[0][0])

Aha: in version 2021 there is no counterpart of the phrase atom 945830, the one which carried `actor=CNH`.

This phrase atom has morphed into a subphrase, and hence we loose the connection and this particular annotation.

### Edge feature "coref"

We also have an edge feature in the module. Let's test that as well.

First we explore the edge feature a little bit.
From which node type to which node type do they go?

We constrain our displays to phrases from now on.

In [19]:
N.displaySetup(condenseType="phrase")
P.displaySetup(condenseType="phrase")

In [20]:
nodeTypes = collections.Counter()

for (f, ts) in P.api.E.coref.items():
    fromType = P.api.F.otype.v(f)
    for t in ts:
        toType = P.api.F.otype.v(t)
        nodeTypes[(fromType, toType)] += 1

In [21]:
nodeTypes

Counter({('word', 'subphrase'): 471,
         ('word', 'phrase_atom'): 20254,
         ('word', 'word'): 19884,
         ('phrase_atom', 'phrase_atom'): 34404,
         ('phrase_atom', 'subphrase'): 1621,
         ('phrase_atom', 'word'): 20254,
         ('subphrase', 'word'): 471,
         ('subphrase', 'subphrase'): 1086,
         ('subphrase', 'phrase_atom'): 1621})

The *coref* relation seems to be symmetrical, so when we check cases, we can skip a number
of pairs.

In [22]:
done = set()

for (fromType, toType) in nodeTypes:
    if (fromType, toType) in done:
        continue
    done.add((fromType, toType))
    done.add((toType, fromType))
    print(f"{fromType:<15} - {toType:<15}")
    template = f"""
{fromType}
-coref> {toType}
"""
    resultsN = N.search(template)
    resultsP = P.search(template)
    
    goodOnes = []
    badOnes = []

    phonoN = lambda n: N.api.T.text(n, fmt="text-phono-full")
    phonoP = lambda n: P.api.T.text(n, fmt="text-phono-full")

    for ((fN, tN), (fP, tP)) in zip(resultsN, resultsP):
        fNp = phonoN(fN)
        fPp = phonoP(fP)
        tNp = phonoN(tN)
        tPp = phonoP(tP)
        if fNp == fPp and tNp == tPp:
            goodOnes.append(f"{fNp} => {tNp}")
        else:
            fDif = fNp if fNp == fPp else f"{fNp} != {fPp}"
            tDif = tNp if tNp == tPp else f"{tNp} != {tPp}"
            badOnes.append((f"{fDif} => {tDif}", fN, fP, tN, tP))
    print(f"good: {len(goodOnes):>5}\nbad : {len(badOnes):>5}")
    if len(goodOnes):
        print("Good:")
        for rep in goodOnes[0:3]:
            print(f"\t{rep}")
    if len(badOnes):
        print("Bad:")
        for (rep, fN, fP, tN, tP) in badOnes[0:3]:
            print(f"\t{rep} {fN} {fP} => {tN} {tP}")
    print("-" * 40)
    print("")

word            - subphrase      
  0.17s 471 results
  0.15s 471 results
good:   471
bad :     0
Good:
	bānˈāʸw  => ʔˈel-ʔahᵃrˈōn 
	zivḥêhem  => bᵊnˈê yiśrāʔˈēl 
	zivḥêhem  => mibbᵊnˈê yiśrāʔˈēl 
----------------------------------------

word            - phrase_atom    
  0.33s 20188 results
  0.30s 20254 results
good:  3785
bad : 16403
Good:
	ʔᵃlêhˈem  => ʔˈel-ʔahᵃrˈōn wᵊʔel-bānˈāʸw wᵊʔˌel kol-bᵊnˈê yiśrāʔˈēl 
	hᵉvîʔˌô  => šˌôr ʔô-ḵˈeśev ʔô-ʕˌēz 
	ʕammˈô .  => ʔˌîš ʔîš 
Bad:
	zzarʕˈô  => ʔˈîš ʔîš  != ʔˈîš  64423 64422 => 944121 944096
	zzarʕˈô  => yittˈēn  != ʔîš  64423 64422 => 944127 944097
	zzarʕˈô  => yûmˈāṯ  != yittˈēn  64423 64422 => 944131 944103
----------------------------------------

word            - word           
  0.43s 19884 results
  0.42s 19884 results
good: 19884
bad :     0
Good:
	zivḥêhem  => zivḥêhˈem 
	zivḥêhem  => lāhˌem 
	zivḥêhem  => ḏōrōṯˈām . 
----------------------------------------

phrase_atom     - phrase_atom    
  0.33s 34215 results
  0.33s 34404 

#### Observations:

All coref links between words and subphrases match perfectly.

But where phrase atoms are involved, we get bad ones, sometimes more bad ones than good ones.

We inspect a few bad cases.

##### between words and phrase atoms:

```
zzarʕˈô  => ʔˈîš ʔîš  != ʔˈîš  64423 64422 => 944121 944096
```

In [23]:
fP = 64422
tP = 944096
pfP = P.api.L.u(fP, otype="phrase")[0]
ptP = P.api.L.u(tP, otype="phrase")[0]
highlightsP = {fP: "orange", tP: "cyan"}

In [24]:
fN = 64423
tN = 944121
pfN = N.api.L.u(fN, otype="phrase")[0]
ptN = N.api.L.u(tN, otype="phrase")[0]
highlightsN = {fN: "orange", tN: "cyan"}

In [25]:
# original coref link
P.pretty(pfP, highlights=highlightsP)
if pfP != ptP:
    P.pretty(ptP, highlights=highlightsP)

In [26]:
# mapped coref link
N.pretty(pfN, highlights=highlightsN)
if pfN != pfP:
    N.pretty(ptN, highlights=highlightsN)

Force majeure! The phrase atom in the original has changed. In the new version it is combined with its neighbour,
and the two constituting parts are now subphrases.

##### between phrase atoms:

```
ʔˌîš ʔˈîš  != ʔˌîš  => ʔˌîš ʔˈîš  != ʔˈîš  943311 943285 => 943311 943286
```

In [27]:
fP = 943285
tP = 943286
pfP = P.api.L.u(fP, otype="phrase")[0]
ptP = P.api.L.u(tP, otype="phrase")[0]
highlightsP = {fP: "orange", tP: "cyan"}

In [28]:
fN = 943311
tN = 943311
pfN = N.api.L.u(tN, otype="phrase")[0]
ptN = N.api.L.u(tN, otype="phrase")[0]
highlightsN = {fN: "orange", tN: "cyan"}

In [29]:
P.pretty(pfP, highlights=highlightsP)
if pfP != ptP:
    P.pretty(ptP, highlights=highlightsP)

In [30]:
N.pretty(pfN, highlights=highlightsN)
if pfN != ptN:
    N.pretty(ptN, highlights=highlightsN)

The same kind of force majeure. 
In this case the link was between the two original phrase atoms.
In the new version these have merged into one phrase atom, and now there is 
a *coref* self-link!

##### between phrase atoms and subphrases:

```
ʔˌîš ʔˈîš  != ʔˌîš  => ʔîš  943311 943285 => 1317262 1317261
```

In [31]:
fP = 943285
tP = 1317261
pfP = P.api.L.u(fP, otype="phrase")[0]
ptP = P.api.L.u(tP, otype="phrase")[0]
highlightsP = {fP: "orange", tP: "cyan"}

In [32]:
fN = 943311
tN = 1317262
pfN = N.api.L.u(fN, otype="phrase")[0]
ptN = N.api.L.u(tN, otype="phrase")[0]
highlightsN = {fN: "orange", tN: "cyan"}

In [33]:
# original coref link
P.pretty(pfP, highlights=highlightsP)
if pfP != ptP:
    P.pretty(ptP, highlights=highlightsP)

In [34]:
# mapped coref link
N.pretty(pfN, highlights=highlightsN)
if pfN != ptN:
    N.pretty(ptN, highlights=highlightsN)

The same kind of force majeure. 

Clearly, there is a massive reorganization of phrase atoms in version `2021` as compared to version `c`.

## Conclusion

It is great to be able to upgrade features from a version against which they have been created to a newer
version.
But the corpus may have been changed in unforeseen ways, and not every node in the old corpus can be necessarily
matched with a unique node in the new corpus.
If there are annotations on such nodes, then they either do not carry over to the new version, or they may carry
over to unintended extra nodes in the new version.

We saw a lot of "bad" cases. But yet, all these discrepancies are really not that bad.
The mapping has always picked the closest node in the new version that corresponds with the original node in the old version.

There are ways to detect such discrepancies, and the node mapping already has relevant information about the quality of the mapping.
In fact, the `migrateFeatures` of Text-Fabric uses the quality information when it assigns feature values to nodes.

But nothing beats generating the features against the new version by the same code that generated them against
the old version.
If there are issues due to important version differences, the author of the generated feature knows best
how to handle that.

# All steps

* **[start](start.ipynb)** your first step in mastering the bible computationally
* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[export](export.ipynb)** export your dataset as an Emdros database
* **[annotate](annotate.ipynb)** annotate plain text by means of other tools and import the annotations as TF features
* **map** map somebody else's annotations to a new version of the corpus
* **[volumes](volumes.ipynb)** work with selected books only
* **[trees](trees.ipynb)** work with the BHSA data as syntax trees

CC-BY Dirk Roorda